In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
import catboost as cb
import catboost.utils as cbu
import hyperopt
import sys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [ ]:
'''
nan_dict = {}
for i in range(len(train.index)) :
    nan_dict[i] = train.iloc[i].isnull().sum()
'''

In [ ]:
'''
nan_dict_seleted = {}
for k, v in sorted(nan_dict.items(), key=lambda item: item[1], reverse = True):
    if v > 130:
        nan_dict_seleted[k] = v
print('--------------' + str(len(nan_dict_seleted))+ '----------------')
'''

In [ ]:
'''
death_0 = 0
death_1 = 0
nan_dict_seleted_remove = []
for i in nan_dict_seleted:
    if train.iloc[i].hospital_death == 0: 
        death_0 = death_0 + 1
        nan_dict_seleted_remove.append(i)
    if train.iloc[i].hospital_death == 1 : death_1 = death_1 + 1
print(death_0)
print(death_1)
print(death_0/death_1)
'''

In [ ]:
#train.shape

In [ ]:
#train.drop(nan_dict_seleted_remove,inplace=True)

In [ ]:
#train.shape

In [ ]:
#train['apache_4a_hospital_death_prob'].fillna(train['apache_4a_hospital_death_prob'].median(), inplace=True)
#test['apache_4a_hospital_death_prob'].fillna(test['apache_4a_hospital_death_prob'].median(), inplace=True)

In [ ]:
#from sklearn.utils import class_weight
#cw = list(class_weight.compute_class_weight('balanced',np.unique(train['hospital_death']),train['hospital_death']))

In [ ]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

In [ ]:
ignore_cat = ['hospital_death','gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']
new_cat_cols = []
'''
for col in train_numerical_cols:
    if train[col].nunique() < 3 and col not in ignore_cat:
        new_cat_cols.append(col)
print(new_cat_cols)
print(len(new_cat_cols))
'''

In [ ]:
'''
def replace_missing_value(df, number_features):
    imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    df_num = df[number_features]
    imputer.fit(df_num)
    print(df[number_features].head())
    df[number_features] = imputer.transform(df_num)
    return df
'''

In [ ]:
#train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].isna().sum()

In [ ]:
'''
train.loc[train['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1
train.loc[train['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1
test.loc[test['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1
test.loc[test['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1
'''


In [ ]:
#train[train['apache_4a_hospital_death_prob'] ==-1]['apache_4a_hospital_death_prob'].count()

In [ ]:
#train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']] = train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].fillna(-1)
#test[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']] = test[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].fillna(-1)

In [ ]:
'''
ahd_imp=SimpleImputer(missing_values=-1,strategy="median" )
ahd_imp.fit(train[["apache_4a_hospital_death_prob"]])
train["apache_4a_hospital_death_prob"]=ahd_imp.transform(train[["apache_4a_hospital_death_prob"]])
test["apache_4a_hospital_death_prob"]=ahd_imp.transform(test[["apache_4a_hospital_death_prob"]])
'''

In [ ]:
'''
ahd_imp=SimpleImputer(missing_values=-1,strategy="median" )
ahd_imp.fit(train[["apache_4a_icu_death_prob"]])
train["apache_4a_icu_death_prob"]=ahd_imp.transform(train[["apache_4a_icu_death_prob"]])
test["apache_4a_icu_death_prob"]=ahd_imp.transform(test[["apache_4a_icu_death_prob"]])
'''

In [ ]:
'apache_4a_hospital_death_prob' in train_numerical_cols

In [ ]:
cols_to_remove = [
    'apache_post_operative','d1_albumin_max','d1_albumin_min','d1_bilirubin_max','d1_bilirubin_min','d1_bun_max','d1_bun_min','d1_creatinine_max',
    'd1_creatinine_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_glucose_max','d1_hemaglobin_min','d1_hematocrit_max','d1_hematocrit_min',
    'd1_mbp_noninvasive_max','d1_mbp_noninvasive_min','d1_platelets_min','d1_sodium_min','d1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','d1_wbc_max','d1_wbc_min',
    'h1_albumin_min','h1_arterial_pco2_max','h1_arterial_pco2_min','h1_arterial_ph_min','h1_bilirubin_max','h1_bilirubin_min','h1_bun_max','h1_bun_min','h1_calcium_min',
    'h1_creatinine_max','h1_creatinine_min', 'h1_diasbp_noninvasive_max','h1_diasbp_noninvasive_min','h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max',
    'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min','h1_lactate_max','h1_lactate_min','h1_mbp_noninvasive_max','h1_mbp_noninvasive_min',
    'h1_pao2fio2ratio_min','h1_platelets_max','h1_platelets_min','h1_potassium_min','h1_sodium_min', 'h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min','h1_wbc_max','h1_wbc_min',
    'paco2_for_ph_apache','readmission_status'
    ]

'''
cols_to_remove = [
    'd1_albumin_max', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max','d1_bun_min','d1_creatinine_max',
    'd1_creatinine_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min', 'd1_hematocrit_max', 'd1_mbp_noninvasive_max','d1_mbp_noninvasive_min', 'd1_platelets_min',
    'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','d1_wbc_max', 'h1_albumin_min', 'h1_arterial_pco2_min','h1_bilirubin_max','h1_bilirubin_min','h1_bun_min',
    'h1_calcium_min',
    'h1_creatinine_max','h1_creatinine_min', 'h1_diasbp_noninvasive_max','h1_diasbp_noninvasive_min','h1_glucose_min', 'h1_hco3_min', 'h1_hemaglobin_min', 'h1_hematocrit_min',
    'h1_inr_max', 'h1_inr_min', 'h1_lactate_min', 'h1_mbp_noninvasive_max','h1_mbp_noninvasive_min', 'h1_platelets_min', 'h1_sodium_min', 
    'h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min', 'h1_wbc_min', 'paco2_for_ph_apache','readmission_status'
]
'''
cols_to_remove = [
    'h1_inr_max', 'h1_inr_min', 'paco2_for_ph_apache','readmission_status',
    'd1_bilirubin_max','d1_creatinine_max','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_mbp_noninvasive_min',
    'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','h1_albumin_min','h1_bilirubin_min','h1_bun_min',
    'h1_creatinine_min','h1_hco3_min','h1_mbp_noninvasive_min','h1_platelets_min','h1_sysbp_noninvasive_max','h1_wbc_min'
]

In [ ]:
len(cols_to_remove)

In [ ]:
#np.setdiff1d(cols_to_remove, train_numerical_cols)

In [ ]:
#np.setdiff1d(cols_to_remove, train_categorical_cols)

In [ ]:
#print(np.setdiff1d(new_cat_cols, cols_to_remove))
#print(np.setdiff1d(new_cat_cols, train_categorical_cols))

In [ ]:
#train[['encounter_id','apache_4a_hospital_death_prob']].head(5)

In [ ]:
train['gcs'] = train[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)
test['gcs'] = test[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)
#train['gcs'] = train[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].sum(axis =1)
#test['gcs'] = test[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].sum(axis =1)

In [ ]:
#train.loc[:,["gcs"]].head()

In [ ]:
#train[train['gcs_unable_apache']==1].shape

In [ ]:
'''
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Cardiovascular')]['age'] = 67
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Gastrointestinal')]['age'] = 72
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Cardiovascular')]['age'] = 67
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Genitourinary')]['age'] = 70
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Gynecological')]['age'] = 34
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Hematological')]['age'] = 70
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Metabolic')]['age'] = 60
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Musculoskeletal/Skin')]['age'] = 64
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Neurological')]['age'] = 70
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Respiratory')]['age'] = 68
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Sepsis')]['age'] = 70
train[train['age'].isna() & (train['apache_3j_bodysystem']=='Trauma')]['age'] = 75
'''

In [ ]:
#pd.crosstab(train['hospital_death'], train['age'])

In [ ]:
#pd.crosstab(train.hospital_death,train[train['gcs'].isna()]['gcs'])

In [ ]:
train['gcs'].fillna(train['gcs'].mean(), inplace = True)
test['gcs'].fillna(test['gcs'].mean(), inplace = True)
#train['gcs'].value_counts()[0]

In [ ]:
#train = replace_missing_value(train, train_numerical_cols)
#train[test_numerical_cols].apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
#test[test_numerical_cols].fillna(test[test_numerical_cols].mean(), inplace=True)
#test[test_numerical_cols].apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
#test[test_numerical_cols].isna().any()

In [ ]:
train.drop(train.loc[train['hospital_admit_source']=='Observation'].index, inplace=True)
#train.drop(train.loc[train['gender'].isna()].index, inplace=True)

In [ ]:
'''
for col in train_categorical_cols:
    print('\n' + col)
    print(train[col].unique())
    print(test[col].unique())
'''

In [ ]:
#train.shape

In [ ]:
100 - (7907 / 91678) * 100

In [ ]:
#train.loc[train['hospital_death']== 1].count()
#91678  - 7907
83771 / 7907

In [ ]:
#train[train['apache_2_bodysystem'] == 'Undefined Diagnoses'].count()

In [ ]:
#train['hospital_admit_source'].unique()

In [ ]:
#train[train['apache_2_bodysystem'] == 'Undefined diagnoses'].count()
train.loc[train['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'
test.loc[test['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'

In [ ]:
train['apache_2_bodysystem'] = train['apache_2_bodysystem'].fillna("Undefined diagnoses")
test['apache_2_bodysystem'] = test['apache_2_bodysystem'].fillna("Undefined diagnoses")

In [ ]:
eth = 'Other/Unknown'
#eth = 'Caucasian'
train['ethnicity'] = train['ethnicity'].fillna(eth)
test['ethnicity'] = test['ethnicity'].fillna(eth)

In [ ]:
train['hospital_admit_source'] = train['hospital_admit_source'].fillna("Other")
test['hospital_admit_source'] = test['hospital_admit_source'].fillna("Other")

In [ ]:
train['icu_admit_source'] = train['icu_admit_source'].fillna("Other ICU")
test['icu_admit_source'] = test['icu_admit_source'].fillna("Other ICU")

In [ ]:
train['gender'] = train['gender'].fillna("NA")
test['gender'] = test['gender'].fillna("NA")

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("NA")
test[test_categorical_cols] = test[test_categorical_cols].fillna("NA")

In [ ]:
#train[train_numerical_cols] = train[train_numerical_cols].fillna(train[train_numerical_cols].median())
#test[test_numerical_cols] = test[test_numerical_cols].fillna(test[test_numerical_cols].median())


In [ ]:
#train[train_numerical_cols].isna().any()

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]

In [ ]:
inputcols.remove('gcs_eyes_apache')
inputcols.remove('gcs_motor_apache')
inputcols.remove('gcs_unable_apache')
inputcols.remove('gcs_verbal_apache')
#inputcols.remove('icu_id')

In [ ]:
len(inputcols)

In [ ]:
'''
cols_to_remove.append('h1_inr_max')
cols_to_remove.append('h1_inr_min')
cols_to_remove.append('paco2_for_ph_apache')
cols_to_remove.append('d1_bilirubin_max')
cols_to_remove.append('d1_bilirubin_min')
'''

In [ ]:

for col in cols_to_remove:
    inputcols.remove(col)


In [ ]:
len(inputcols)

In [ ]:
for col in new_cat_cols:
    if col in inputcols:
        print('---------------'+ col+ '---------------')
        if(train[col].isna().sum()==0):
            train_categorical_cols.append(col)

In [ ]:
#train_categorical_cols.remove('ethnicity')
#test_categorical_cols.remove('ethnicity')
#inputcols.remove('ethnicity')

In [ ]:
'ethnicity' in inputcols

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
                   #group_id = ['encounter_id']
                 )

In [ ]:
#xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=.8,random_state=1234)
#model = CatBoostClassifier(learning_rate=0.08166369766256530,l2_leaf_reg = 8.836837790286535, loss_function='Logloss',custom_metric= ['AUC'],eval_metric='Accuracy',use_best_model=True,random_seed=42)
#model.fit(xtrain,ytrain,cat_features=train_categorical_cols,eval_set=(xtest,ytest))

In [ ]:
#print('Tree count: ' + str(model.tree_count_))

In [ ]:
#model.get_params()

In [ ]:
#clf.get_best_score()['learn']['Logloss']

In [ ]:
# 83773/7903 # 83773/7903

In [ ]:
'''
for i in nan_dict_seleted:
    print(train.iloc[i].hospital_death)
'''

In [ ]:
#train.iloc[3011]

In [ ]:

predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
lcount = 0
for i in range(60):
    print('-------------' + str(i) + '----------------------')
    clf = CatBoostClassifier(learning_rate=0.08166369766256530, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.836837790286535,task_type="CPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=False,silent=True)
    #print('Tree count: ' + str(clf.tree_count_))
    #print(clf.get_best_score())
    #if clf.get_best_score()['learn']['Logloss'] < 0.0599:
    #    lcount = lcount + 1
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
print(lcount)    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1910.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019

In [ ]:
print('completed')

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
lcount = 0
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08166369766256530005, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.836837790286535,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    print(clf.get_best_score())
    if clf.get_best_score()['learn']['Logloss'] < 0.0595:
        lcount = lcount + 1
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
print(lcount)    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1906.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
print('completed')

In [ ]:
'''
params = {}
params['loss_function'] = 'Logloss'
params['iterations'] = 551
params['custom_loss'] = 'AUC'
params['random_seed'] = 63
params['learning_rate'] = 0.5

cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=train_categorical_cols),
    fold_count = 5,
    type = 'Classical',
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)
'''

In [ ]:
'''
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, stratified: {:.4f} ± {:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)
'''

In [ ]:
cv_data

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391



#0.07260908767
for i in range(1):
    clf = CatBoostClassifier(learning_rate=0.062338418743052904, depth = 5,loss_function='Logloss', eval_metric = 'Accuracy', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 6.706159252357066,task_type="GPU", random_seed=20181224, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True, use_best_model = True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1801.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08166369766256530, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.836837790286535,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1504.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08166369766256534, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.835837790286535,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1102.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08166369766256534, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.836837790286535,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1103.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08166369766256530, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 8.835837790286535,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("1104.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
### hyperparameter tuning example grid for catboost : 
'''
grid = {'learning_rate': [0.01, 0.04],
        'depth': [10],
        #'l2_leaf_reg': [3,4,5],
        "iterations": [500],
        "custom_metric":['Logloss', 'AUC']}

model = CatBoostClassifier(task_type="GPU")

## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                       train_pool,
                                       plot=True,
                                       refit = True, #  refit best model on all data
                                      partition_random_seed=42)

print(model.get_best_score())
print("best model params: \n",grid_search_result["params"])
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08000100126274019, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0901.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08000100126274019, boosting_type='Ordered', depth = 10,loss_function='Logloss', eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0902.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function='Logloss', eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0217.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

In [ ]:
'''
class UciAdultClassifierObjective(object):
    def __init__(self, dataset, const_params, fold_count):
        self._dataset = dataset
        self._const_params = const_params.copy()
        self._fold_count = fold_count
        self._evaluated_count = 0
        
    def _to_catboost_params(self, hyper_params):
        return {
            'learning_rate': hyper_params['learning_rate'],
            'depth': hyper_params['depth'],
            'l2_leaf_reg': hyper_params['l2_leaf_reg']}
    
    # hyperopt optimizes an objective using `__call__` method (e.g. by doing 
    # `foo(hyper_params)`), so we provide one
    def __call__(self, hyper_params):
        # join hyper-parameters provided by hyperopt with hyper-parameters 
        # provided by the user
        params = self._to_catboost_params(hyper_params)
        params.update(self._const_params)
        
        print('evaluating params={}'.format(params), file=sys.stdout)
        sys.stdout.flush()
        
        # we use cross-validation for objective evaluation, to avoid overfitting
        scores = cb.cv(
            pool=self._dataset,
            params=params,
            fold_count=self._fold_count,
            partition_random_seed=20181224,
            verbose=False)
        
        # scores returns a dictionary with mean and std (per-fold) of metric 
        # value for each cv iteration, we choose minimal value of objective 
        # mean (though it will be better to choose minimal value among all folds)
        # because noise is additive
        max_mean_auc = np.max(scores['test-AUC-mean'])
        print('evaluated score={}'.format(max_mean_auc), file=sys.stdout)
        
        self._evaluated_count += 1
        print('evaluated {} times'.format(self._evaluated_count), file=sys.stdout)
        
        # negate because hyperopt minimizes the objective
        return {'loss': -max_mean_auc, 'status': hyperopt.STATUS_OK}
    
def find_best_hyper_params(dataset, const_params, max_evals=100):    
    # we are going to optimize these three parameters, though there are a lot more of them (see CatBoost docs)
    parameter_space = {
        'learning_rate': hyperopt.hp.uniform('learning_rate', 0.01, 0.1),
        'depth': hyperopt.hp.randint('depth', 12),
        'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 10)}
    objective = UciAdultClassifierObjective(dataset=dataset, const_params=const_params, fold_count=6)
    trials = hyperopt.Trials()
    best = hyperopt.fmin(
        fn=objective,
        space=parameter_space,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        rstate=np.random.RandomState(seed=20181224))
    return best

def train_best_model(X, y, const_params, max_evals=100, use_default=False):
    # convert pandas.DataFrame to catboost.Pool to avoid converting it on each 
    # iteration of hyper-parameters optimization
    dataset = Pool(X, y, cat_features=train_categorical_cols)
    
    if use_default:
        # pretrained optimal parameters
        best = {
            'learning_rate': 0.4234185321620083, 
            'depth': 5, 
            'l2_leaf_reg': 9.464266235679002}
    else:
        best = find_best_hyper_params(dataset, const_params, max_evals=max_evals)
    
    # merge subset of hyper-parameters provided by hyperopt with hyper-parameters 
    # provided by the user
    hyper_params = best.copy()
    hyper_params.update(const_params)
    
    # drop `use_best_model` because we are going to use entire dataset for 
    # training of the final model
    hyper_params.pop('use_best_model', None)
    
    model = cb.CatBoostClassifier(**hyper_params)
    model.fit(dataset, verbose=False)
    
    return model, hyper_params


# make it True if your want to use GPU for training
have_gpu = False
# skip hyper-parameter optimization and just use provided optimal parameters
use_optimal_pretrained_params = False
# number of iterations of hyper-parameter search
hyperopt_iterations = 15

const_params = dict({
    'task_type': 'GPU' if have_gpu else 'CPU',
    'loss_function': 'Logloss',
    'eval_metric': 'AUC', 
    'custom_metric': ['AUC'],
    'iterations': 1000,
    'random_seed': 20181224})

model, params = train_best_model(
    X, y, 
    const_params, 
    max_evals=hyperopt_iterations, 
    use_default=use_optimal_pretrained_params)
print('best params are {}'.format(params), file=sys.stdout)
'''

In [ ]:
print('hyper completed')

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391
random_seed = 20181224

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08577070126274019,depth = 10,loss_function='Logloss', 
                             eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, 
                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0212.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

In [ ]:
'''
predictions = []
scale_pos_weight = 10.594536486657391
random_seed = 20181224

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function='Logloss', 
                             eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, 
                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0210.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

In [ ]:
print('completed')